### Generators

As we saw in the lecture, generator expressions are a way to create generator objects using comprehension syntax (there are other ways too, using `yield` in functions, but that's beyond the scope of this course.)

Generators are iterators - so they have an `__iter__` method (which just returns the object itself), and a `__next__` method used to request the next object while iterating. And they also raise a `StopIteration` once they reach the end of the iteration.

In [1]:
squares = (i ** 2 for i in range(5))

And, just like iterators, they are one-time use only:

In [2]:
for i in squares:
    print(i)

0
1
4
9
16


And `squares` is now an exhausted iterator:

In [3]:
for i in squares:
    print('iterating again...')

As you can see, nothing happened!

This is contrast to what happens with a list comprehension:

In [4]:
l = [i ** 2 for i in range(5)]

In [5]:
for i in l:
    print(i)

0
1
4
9
16


In [6]:
for i in l:
    print(i)

0
1
4
9
16


If we want to re-iterate over squares, we'll have to re-create the generator:

In [7]:
squares = (i ** 2 for i in range(5))

Remember when I told you that `ist(<iterable>)` essentially built a list by iterating over the iterable?

If you pass a generator (or any iterator) to the list() or tuple() functions, you **will exhaust** the iterator!

In [8]:
squares = (i ** 2 for i in range(5))

In [9]:
list(squares)

[0, 1, 4, 9, 16]

In [10]:
list(squares)

[]

We can verify that generators are iterators by examining the `iter` and `next` methods:

In [11]:
squares = (i ** 2 for i in range(5))

In [12]:
iter(squares) is squares

True

So, calling `iter()` on a generator just returns the generator itself - consistent with what we have seen with iterators.

And we can call `next()` on the generator, until we get a `StopIteration` exception::

In [13]:
try:
    while True:
        print(next(squares))
except StopIteration:
    print('finished iterating')

0
1
4
9
16
finished iterating


And of course, calling next() again will just result in the same `StopIteration` exception - once the iterator says it's done, it's done!

In [14]:
try:
    next(squares)
except StopIteration:
    print('no more!')

no more!


You have to be extra careful with generators, and iterators in general, as you can easily consume or exhaust the iterator by accident!

In [15]:
squares = (i ** 2 for i in range(5))

In [16]:
3 in squares

False

In [17]:
list(squares)

[]

Why do you think that happened?

Remember our discussion on membership testing in lists vs sets - for lists Python has to iterate through the iterable until it finds (or not) the item.

That's exactly what happened here. In fact, it could get worse:

In [18]:
squares = (i ** 2 for i in range(5))

list(squares)

[0, 1, 4, 9, 16]

In [19]:
squares = (i ** 2 for i in range(5))

4 in squares

True

In [20]:
list(squares)

[9, 16]

As you can see, when Python did the membership test, it found `4`, so it stopped iterating through `squares` and returned `True` - this means we actually two elements left over in the iterator! 

This can cause some very weird bugs!

Let's examine something we talked about in the lecture: generators are very quick to create compared to a list comprehension, since the collection elements are not actually created until they are requested - and then they are calculated and handed off one by one (lazy iteration):

In [21]:
from timeit import timeit

In [22]:
timeit('[i ** 2 for i in range(25_000_000)]', number=1)

2.9994857999990927

In [23]:
timeit('(i ** 2 for i in range(25_000_000))', number=1)

1.8499995348975062e-05

And in fact, for the generator it does not matter how many elements would be produced:

In [24]:
timeit('(i ** 2 for i in range(100_000_000_000))', number=1)

6.000002031214535e-06

In summary, generators are great! But beware, they are not re-usable, and if you're going to need to iterate through them multiple times, you may be better off making the performance/memory trade-off.